In [13]:
import os
import shutil
from pathlib import Path

import multiprocessing as mp
from pathlib import Path
import os
import json
import pandas as pd
from tqdm import tqdm
from lhotse import CutSet, RecordingSet, SupervisionSet, MonoCut
from lhotse.cut import append_cuts
from lhotse.recipes import prepare_tedlium

import logging

logging.basicConfig(level=logging.INFO)

In [14]:
IN_DIR = "../datasets/LongSpeechSource/TEDLIUM_release-3"
OUT_DIR = '../datasets/LongSpeech'

In [16]:
manifests = prepare_tedlium(tedlium_root=IN_DIR, output_dir=OUT_DIR, num_jobs=15)

Scanning audio files (*.sph): 2351it [00:18, 128.25it/s]
Scanning audio files (*.sph): 8it [00:00, 36.16it/s]
Scanning audio files (*.sph): 11it [00:00, 55.54it/s]


In [22]:
cuts = CutSet()
for part in manifests.keys():
    rs = manifests[part]['recordings']
    ss = manifests[part]['supervisions']
    cut = CutSet.from_manifests(recordings=rs, supervisions=ss)
    cuts += cut

In [23]:
cuts.to_jsonl(os.path.join(OUT_DIR, 'raw_ted_lium_cuts.jsonl'))

In [3]:
def collect_subset(
 tedlium_root: Path,
 output_dir: Path,
 num_files: int = 1
):
    # 原始路径
    sph_dir = Path(os.path.join(tedlium_root, "data", "sph"))
    stm_dir = Path(os.path.join(tedlium_root, "data", "stm"))
    out_sph_dir = Path(os.path.join(output_dir, "data", "sph"))
    out_stm_dir = Path(os.path.join(output_dir, "data", "stm"))
    out_sph_dir.mkdir(parents=True, exist_ok=True)
    out_stm_dir.mkdir(parents=True, exist_ok=True)

    # 找到所有 .sph 文件
    sph_files = sorted(sph_dir.glob("*.sph"))
    selected = sph_files[:num_files]
    print(sph_dir)
    for sph_path in selected:
        base = sph_path.stem  # TED_2008_0001
        stm_path = stm_dir / f"{base}.stm"

        if not stm_path.exists():
            print(f"⚠️ STM not found for: {base}")
            continue

        # 复制 .sph 和对应 .stm
        shutil.copy(sph_path, out_sph_dir / sph_path.name)
        shutil.copy(stm_path, out_stm_dir / stm_path.name)

        print(f" Copied: {base}.sph and {base}.stm")

In [4]:
collect_subset(INPUT_FOLDER, OUTPUT_FOLDER)

../datasets/LongSpeechSource/TEDLIUM_release-3/data/sph
 Copied: 911Mothers_2010W.sph and 911Mothers_2010W.stm
